# **Clinical Malaria Reinforced Learnning**

In [4]:
#installing Libraries
!pip install stable-baselines3[extra] gym matplotlib pandas scikit-learn shimmy

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 322.4 kB/s eta 0:00:03
   -------------- ------------------------- 0.5/1.5 MB 322.4 kB/s eta 0:00:03
   -------------- ----------

In [5]:
!pip install ipywidgets

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 4.2 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.3 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 5.1 MB/s eta 0:00:00


In [8]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


# Required Libraries

In [32]:

import gym
from gym import spaces
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive, HBox, VBox, Output

#Step 1: Data Preparation

In [33]:
# malaria Dataset
file_path = 'Clinical Main Data.xlsx'
data= pd.read_excel(file_path)


In [34]:
# Preview Data
print(data.head())

     lab_no     sex  age  temperature                Thick Film  Cases  \
0  LT014233    Male   35         37.5  No Malaria Parasite seen      0   
1  LT144344    Male   36         37.5  No Malaria Parasite Seen      0   
2  LT005429  Female   29         36.7                      NMPs      0   
3  LT038865  Female   29         36.1                      NMPs      0   
4  LT095829  Female   30         36.7                      NMPs      0   

   chill_cold  headache  fever  generalized body pain  abdominal pain  \
0       False      True   True                  False           False   
1        True     False   True                   True           False   
2       False      True  False                  False           False   
3       False      True  False                   True           False   
4       False      True   True                  False           False   

   Loss of appetite  joint pain  vomiting  nausea  diarrhea  
0             False       False      True   False     

In [35]:
# Define features and target variable
features = [
    'chill_cold', 'headache', 'fever', 'generalized body pain',
    'abdominal pain', 'Loss of appetite', 'joint pain', 'vomiting',
    'nausea', 'diarrhea'
]
target = 'Cases'

# Convert TRUE/FALSE to 1/0 for binary classification
data['Cases'] = data['Cases'].astype(int)  # Ensure target variable is numeric
data[features] = data[features].astype(int)  # Convert feature columns to numeric


In [36]:
# Split into train and test sets
X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [37]:
# Save the scaler
scaler_path = "models/scaler.pkl"
os.makedirs("models", exist_ok=True)
joblib.dump(scaler, scaler_path)

['models/scaler.pkl']

# Step 2: Define Custom Environment



In [39]:
class MalariaPredictionEnv(gym.Env):
    def __init__(self, X, y):
        super(MalariaPredictionEnv, self).__init__()
        self.X = X
        self.y = y
        self.current_index = 0

        # Define state and action spaces
        self.observation_space = spaces.Box(low=-1, high=1, shape=(X.shape[1],), dtype=np.float32)
        self.action_space = spaces.Discrete(2)  # Binary classification: 0 (Negative) or 1 (Positive)

    def reset(self):
        self.current_index = 0
        return self.X[self.current_index]

    def step(self, action):
        actual = self.y[self.current_index]
        reward = 1 if action == actual else -1  # Reward for correct prediction
        self.current_index += 1
        done = self.current_index >= len(self.X)
        next_state = self.X[self.current_index] if not done else np.zeros_like(self.X[0])
        return next_state, reward, done, {}

    def seed(self, seed=None):
        np.random.seed(seed)

# Initialize Environment for Training
env = MalariaPredictionEnv(X_train_scaled, y_train.to_numpy())

# Step 3: Train the RL Model

In [40]:
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("models/ppo_malaria")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\victor.idakwo\Documents\ehealth Africa\ehealth Africa\eHA GitHub\openweather-data-pipeline\venv\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 2018 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 968         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.018972268 |
|    clip_fraction        | 0.259       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.68       |
|    explained_variance   | -0.0283     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.33        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.032      |
|    value_loss           | 8.09        |
-----------------------------------------
----------------------------------

# Step 4: Load the Model for Continuous Training and Predictions

In [41]:
model = PPO.load("models/ppo_malaria")
scaler = joblib.load(scaler_path)


# Step 5: Create User Input Interface with Widgets

In [45]:
feature_widgets = {
    feature: widgets.ToggleButton(description=feature.replace('_', ' ').title(), value=False)
    for feature in features
}

# Input for actual cases
cases_input = widgets.IntText(value=0, description='Actual Cases:', min=0)

# Output widget for displaying predictions
output = Output()

# Function to handle the input and make a prediction
def handle_submit(change):
    with output:
        output.clear_output()

        # Gather user input
        selected_features = [int(btn.value) for btn in feature_widgets.values()]
        actual_cases = cases_input.value

        # Scale the new input data
        new_data = np.array([selected_features])
        new_scaled_data = scaler.transform(new_data)

        # Predict using the trained model
        prediction = model.predict(new_scaled_data)[0]

        # Convert prediction to a label
        predicted_case = "Positive (1)" if prediction == 1 else "Negative (0)"
        actual_case = "Positive (1)" if actual_cases == 1 else "Negative (0)"

        # Display results
        print("\nSelected Features:")
        for key, value in zip(features, selected_features):
            print(f"{key.replace('_', ' ').title()}: {'Yes' if value else 'No'}")

        print(f"\nPredicted Case: {predicted_case}")
        print(f"Actual Case: {actual_case}")

# Create the submit button and link to the function
submit_button = widgets.Button(description="Submit", layout=widgets.Layout(width="30%", height="40px"))
submit_button.on_click(handle_submit)

# Layout the interface
feature_title = widgets.Label("Select Symptoms (Toggle for Yes/No)")
feature_layout = VBox([
    HBox([feature_widgets[feature] for feature in list(feature_widgets.keys())[i:i+5]])
    for i in range(0, len(feature_widgets), 5)
])

ui = VBox([
    feature_title,
    feature_layout,
    HBox([cases_input]),
    submit_button
])

# Display the UI
display(ui, output)


Output()